#### Terminal Access

In [2]:
from src import *
# chat_with_llama("""""", use_tool=True)

In [6]:
import os
os.environ["SERP_API_KEY"] = "5affe20ad27423ecb9954ffa3151680c42adadca"

In [5]:
# OverAll: Search for Snake Game implementation online -> write content into local script -> run & debug & test
# 1. Web Search + Voyager
out = chat_with_llama("""Search the web for latest tweet from Sam Altman""", use_tool=True)
# 2. 


User message: Search the web for latest tweet from Sam Altman
Web Search Tool


In [16]:
out[7]

{'title': 'sam altman twitter - The Economic Times - IndiaTimes',
 'link': 'https://economictimes.indiatimes.com/topic/sam-altman-twitter',
 'snippet': 'sam altman twitter Latest Breaking News, Pictures, Videos, and Special Reports from The Economic Times. sam altman twitter Blogs, Comments and Archive News ...',
 'position': 8}

In [2]:
from src import *

# Each request can NOT be too lenghty, otherwise function call will not work properly

#
# Read
read_out = chat_with_llama("read local file street_fighter.py")
# Run  
run_out = chat_with_llama("run local file street_fighter.py")
# Write
write_in = chat_with_llama("write the next 10 lines of street figher game into local file street_fighter.py")
print(out)

# Loop back the observation into itself and ask for the next step




User message: read local file street_fighter.py
File Reader Tool
print("Ryu: Hadouken!")



In [2]:
out = chat_with_llama("write the next 10 lines of street figher game into local file street_fighter.py")


User message: write the next 10 lines of street figher game into local file street_fighter.py
File Writer Tool


In [8]:
from src.action import *

GROQ_API_KEY = os.environ["GROQ_API_KEY"]
E2B_API_KEY = os.environ["E2B_API_KEY"]
ANTHROPIC_API_KEY = os.environ["ANTHROPIC_API_KEY"]
groq_client = Groq(api_key=GROQ_API_KEY)


user_message = "Write a snake game into a local file named snake_game.py"

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": user_message}
]

MODEL_NAME = "llama3-70b-8192"

response = groq_client.chat.completions.create(
    model=MODEL_NAME,
    messages=messages,
    tools=tools,
    tool_choice="auto",
    max_tokens=4096,
)

response_message = response.choices[0].message

In [2]:
from src import *


request = "run the file snake.py and report the issue"
out = chat_with_llama(request)


has_bug = "Error" in out
if has_bug:
    print("Please debug")
else:
    print("think about the next step to complete {initial_goal}")
    .....

SyntaxError: invalid syntax (1239661712.py, line 13)

In [10]:
# Get Deeper
from src.utils import graph
graph.astream("run the file snake.py and report the issue")

<async_generator object Pregel.astream at 0x123d65d50>

In [5]:
from src import gladia, graph, perception, utils
# Upon the Stop Command this break the loops
while True:
    question = await gladia.listen()
    if "Stop" in question:
        print("Breaking Out!!!!!!")
        break



Connected!
Please ask me a question:
final: (en)  you
Final transcription received: (en)  you
Final Transcription:   you
Connected!
Please ask me a question:
partial: (en)  How is it?
partial: (en)  How is everything?
partial: (en)  How is everything?
final: (en)  How is everything?
Final transcription received: (en)  How is everything?
Final Transcription:   How is everything?
Connected!
Please ask me a question:
partial: (en)  Voila Baud
partial: (en)  What about Don?
partial: (en)  What about Donald Trump?
partial: (en)  What about Donald Trump?
final: (en)  What about Donald Trump?
Final transcription received: (en)  What about Donald Trump?
Final Transcription:   What about Donald Trump?
Connected!
Please ask me a question:
final: (en)  Stop.
Final transcription received: (en)  Stop.
Final Transcription:   Stop.
Breaking Out!!!!!!


In [1]:
from src import gladia, graph, perception, utils
import asyncio
import playwright
from src import gladia, graph, perception, utils
from playwright.async_api import async_playwright
from src.utils import graph
import logging
logger = logging.getLogger("Surfer Agent")
logger.setLevel(logging.INFO)
hander = logging.StreamHandler()
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
hander.setFormatter(formatter)
logger.addHandler(hander)


# Declare the global variables
break_out = False
if_end = False
question = ""
search_iter = 0

# One Concurrent Async Process | Gradia Streaming Audio Input
async def listen_for_stop():
    global break_out, if_end, question
    while True:
        question = await gladia.listen()
        logger.info(f"Getting Question: {question}")
        if "Stop" in question:
            logger.info("Breaking out upon hearing 'Stop'.")
            break_out = True
            print("Breaking Out!!!!!!")
        if "End" in question:
            logger.info("End the Agent upon hearing 'End'.")
            if_end = True
            break

async def web_voyager():
    global break_out, if_end, question, search_iter

    while not if_end:
        search_iter += 1
        print(f"------ \nSearch Iteration: {search_iter} \n------")
        browser = await async_playwright().start()
        browser = await browser.chromium.launch(headless=False, args=None)

        # Change the Starting Point for the Web-Browsing Agent
        page = await browser.new_page()
        _ = await page.goto("https://www.google.com") # This is where we decide the starting point of the agent
        logger.info(f"Navigating to Google on a new page with question: {question}")

        event_stream = graph.astream(
            {
                "page": page,
                "input": question,
                "scratchpad": [],
            },
            {
                "recursion_limit": 50,
            },
        )

        final_answer = None
        steps = []
        async for event in event_stream:
            # We'll display an event stream here
            if "agent" not in event:
                continue
            if break_out:
                break_out = False
                break
            event['agent']['input'] = question # Dummny Slot-in Place for further requests from user
            logger.info(f"Updated Agent Input: {event['agent']['input']}")

            pred = event.get("agent", {}).get("prediction") or {}
            # print("Event: ", event)
            # print("Prediction: \n", pred)
            action = pred.get("action")
            action_input = pred.get("args")
            logger.info(f"Action: \n{len(steps) + 1}. {action}: {action_input}")
            print(f"Action: \n{len(steps) + 1}. {action}: {action_input}")
            steps.append(f"{len(steps) + 1}. {action}: {action_input}")
            if "ANSWER" in action:
                final_answer = action_input[0]
                logger.info(f"Final Answer: {final_answer}")
                print("Final Answer: ", final_answer)
                break


async def main():
    # Create and run the tasks concurrently
    task1 = asyncio.create_task(listen_for_stop())
    task2 = asyncio.create_task(web_voyager())
    await asyncio.gather(task1, task2)

# Run the async main function
# asyncio.run(main())
asyncio.ensure_future(main())

<Task pending name='Task-5' coro=<main() running at /var/folders/bt/rngwf7wj73x2zfr_7chtd0jh0000gn/T/ipykernel_28170/776611990.py:89>>

------ 
Search Iteration: 1 
------
Connected!
How can I help:


2024-05-08 13:09:14,313 - Surfer Agent - INFO - Navigating to Google on a new page with question: 


partial: (en)  Blade
partial: (en)  latest
partial: (en)  latest
partial: (en)  latest tweet
partial: (en)  latest tweet from
partial: (en)  latest tweet from
partial: (en)  Ladies, tweet from Sam
partial: (en)  Ladies, tweet from Sam Oddman
partial: (en)  Ladies, tweet from Sam Oddman
final: (en)  latest tweet from Sam Altman
Final transcription received: (en)  latest tweet from Sam Altman
Final Transcription:   latest tweet from Sam Altman


2024-05-08 13:09:19,845 - Surfer Agent - INFO - Getting Question:  latest tweet from Sam Altman


...

Action: 
3. ANSWER;: ['Mount Everest is the highest mountain in the world, with an elevation of 8,849 meters.']
Final Answer:  Mount Everest is the highest mountain in the world, with an elevation of 8,849 meters.
------ 
Search Iteration: 2 
------


2024-05-08 13:09:45,181 - Surfer Agent - INFO - Navigating to Google on a new page with question:  latest tweet from Sam Altman
2024-05-08 13:09:50,790 - Surfer Agent - INFO - Updated Agent Input:  latest tweet from Sam Altman
2024-05-08 13:09:50,790 - Surfer Agent - INFO - Action: 
1. Type: ['6', 'Sam Altman latest tweet']


Action: 
1. Type: ['6', 'Sam Altman latest tweet']
final: (en)  you
Final transcription received: (en)  you
Final Transcription:   you


2024-05-08 13:10:11,316 - Surfer Agent - INFO - Getting Question:  you


Connected!
How can I help:


In [17]:
import playwright
from src import gladia, graph, perception, utils
from playwright.async_api import async_playwright
from src.utils import graph

# question = await gladia.listen()

async def listen_for_stop():
    global break_out
    while True:
        question = await gladia.listen()
        if "Stop" in question:
            break_out = True
            print("Breaking Out!!!!!!")
            break


while True:
    browser = await async_playwright().start()
    browser = await browser.chromium.launch(headless=False, args=None)

    # Change the Starting Point for the Web-Browsing Agent
    page = await browser.new_page()
    _ = await page.goto("https://gist.github.com/wynand1004/ec105fd2f457b10d971c09586ec44900") # This is where we decide the starting point of the agent
    question = "read and scrape the python code from the page"

    event_stream = graph.astream(
        {
            "page": page,
            "input": question,
            "scratchpad": [],
        },
        {
            "recursion_limit": 50,
        },
    )

    final_answer = None
    steps = []
    async for event in event_stream:
        # We'll display an event stream here
        if "agent" not in event:
            continue
        if break_out:
            break_out = False
            break
        event['agent']['input'] = question # Dummny Slot-in Place for further requests from user
        pred = event.get("agent", {}).get("prediction") or {}
        print("Event: ", event)
        print("Prediction: \n", pred)
        action = pred.get("action")
        action_input = pred.get("args")
        print(f"Action: \n{len(steps) + 1}. {action}: {action_input}")
        steps.append(f"{len(steps) + 1}. {action}: {action_input}")
        if "ANSWER" in action:
            final_answer = action_input[0]
            break

NameError: name 'break_out' is not defined

In [19]:
page.url

'https://gist.github.com/wynand1004/ec105fd2f457b10d971c09586ec44900'

In [10]:
event['agent']['input']

'read and scrape the python code from the page'

In [8]:
event['agent']['prediction']['args']



"Could not parse LLM Output: Thought: The screenshot shows a webpage with Python code related to creating a snake game. The code initializes the game window, creates a snake head, food, and defines a pen for writing the score. The user has asked to read and scrape the Python code from the page.\n\nAction: ANSWER; The Python code in the screenshot is for setting up a simple snake game using the turtle graphics library. It includes the following key parts:\n\n- Importing the turtle module.\n- Setting up the game window with a title, background color, size, and turning off screen updates for performance.\n- Creating the snake's head with a specific shape, color, and initial position.\n- Defining the snake's food with a shape, color, and position.\n- Initializing a pen to write the score, with a specific shape, color, and position.\n- An empty list named 'segments' is defined, presumably for storing the segments of the snake's body as the game progresses.\n\nThis code snippet is likely par

In [10]:
from langchain_core.messages import HumanMessage
graph.get_graph()

Graph(nodes={'__start__': Node(id='__start__', data=<class 'pydantic.v1.main.LangGraphInput'>), '__end__': Node(id='__end__', data=<class 'pydantic.v1.main.LangGraphOutput'>), 'agent': Node(id='agent', data=RunnableLambda(afunc=annotate)
| RunnableAssign(mapper={
    prediction: RunnableLambda(format_descriptions)
                | ChatPromptTemplate(input_variables=['bbox_descriptions', 'img', 'input'], input_types={'scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'scratchpad': []}, messages=[SystemMessagePromptTemplate(prompt=[PromptTemplate(input_variables=[], template="Imagine you are a robot browsing the web, just like humans. Now you need to complete a task. In each iteration, you will receive an Observation th

In [13]:
graph.get_graph().print_ascii()

                                                                     +-----------+                                                                 
                                                                     | __start__ |                                                                 
                                                                     +-----------+                                                                 
                                                                           *                                                                       
                                                                           *                                                                       
                                                                           *                                                                       
                                                                       +-------+*****                           